In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import sys
import config

from tensorflow.keras.models import load_model

sys.path.append('model_building/create_image_folders.py')
from model_building.create_image_folders import * 
sys.path.append('model_building/cnn_model_keras.py')
from model_building.cnn_model_keras import *
sys.path.append('model_building/new_keras_model.py')
from model_building.new_keras_model import *
sys.path.append('model_test/test_model.py')
from model_test.test_model import *

import warnings
warnings.simplefilter("ignore", UserWarning)
import logging
logging.getLogger('tensorflow').disabled = True

## Load Data

In [ ]:
## Set paths
img_folder = os.path.join(os.getcwd(),"data","ai_ready","images")
train_img = os.path.join(os.getcwd(),"data","ai_ready","train_images")
val_img = os.path.join(os.getcwd(),"data","ai_ready","val_images")
test_img = os.path.join(os.getcwd(),"data","ai_ready","test_images")
labels_image = os.path.join(os.getcwd(),"data","ai_ready","x-ai_data.csv")
create_images =False

## Create Subfolder

Use the fonction "subfolders" to do the same thing as the cells below, DO NOT run it twice to avoid duplicates

In [ ]:
labels_image = pd.read_csv(labels_image)
train = labels_image.loc[labels_image['split']=='train']
val = labels_image.loc[labels_image['split']=='validation']

In [ ]:
train0 = train.loc[train['class']==0]
train1 = train.loc[train['class']==1]

val0 = val.loc[val['class']==0]
val1 = val.loc[val['class']==1]


In [ ]:
#Move TRAIN images labeled 0 to the correct folder

for i in train0.index:
    im = cv2.imread(img_folder+train0.loc[i,'filename'])
    cv2.imwrite(train_img + '0/' +train0.loc[i,'filename'], im)
#Move TRAIN images labeled 1 to the correct folder
for i in train1.index:
    im = cv2.imread(img_folder+train1.loc[i,'filename'])
    cv2.imwrite(train_img+'1/' +train1.loc[i,'filename'], im)

In [ ]:
#Move val images labeled 0 to the correct folder
for i in val0.index:
    im = cv2.imread(img_folder+val0.loc[i,'filename'])
    cv2.imwrite(val_img + '0/' +val0.loc[i,'filename'], im)
#Move val images labeled 1 to the correct folder
for i in val1.index:
    im = cv2.imread(img_folder+val1.loc[i,'filename'])
    cv2.imwrite(val_img+'1/' +val1.loc[i,'filename'], im)

In [ ]:
# for i in train0.index:
#     try:

#         im = cv2.imread(os.path.join(img_folder, train0.loc[i, "filename"]))
#         cv2.imwrite(os.path.join(train_img, "0", train0.loc[i, "filename"]), im)
#     except:
#         ipdb.set_trace()
# # Move TRAIN images labeled 1 to the correct folder
# for i in train1.index:
#     im = cv2.imread(os.path.join(img_folder, train1.loc[i, "filename"]))
#     cv2.imwrite(os.path.join(train_img, "1", train1.loc[i, "filename"]), im)

# for i in val0.index:
#     im = cv2.imread(os.path.join(img_folder, val0.loc[i, "filename"]))
#     cv2.imwrite(os.path.join(val_img, "0", val0.loc[i, "filename"]), im)

# # Move TRAIN images labeled 1 to the correct folder
# for i in val1.index:
#     im = cv2.imread(os.path.join(img_folder, val1.loc[i, "filename"]))
#     cv2.imwrite(os.path.join(val_img, "1", val.loc[i, "filename"]), im)

# # Remove hidden file
# try:
#     shutil.rmtree("image/train/.ipynb_checkpoints")
#     shutil.rmtree("image/val/.ipynb_checkpoints")
# except:
#     pass

In [ ]:
# import shutil

# #Remove hidden file
# shutil.rmtree("image/train/.ipynb_checkpoints")
# shutil.rmtree("image/val/.ipynb_checkpoints")

## Model

In [ ]:
## Train and Val dataset
tf.config.list_physical_devices()
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
physical_devices = tf.config.list_physical_devices("GPU")
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
train_ds = train_set(train_img, config.image_size, config.batch_size)
val_ds = val_set(val_img, config.image_size, config.batch_size)

In [ ]:
plt.figure(figsize=(10, 10))
# for images, labels in train_ds.__getitem__(3):
images, labels = train_ds.__getitem__(1)
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title(int(labels[i]))
    plt.axis("off")

In [ ]:
## Model
model = make_model(input_shape=config.image_size + (3,), num_classes=2)

In [ ]:
train_model(model, train_ds, val_ds, 2)

In [ ]:
model.save("model2.h5")

In [ ]:
#visualize data augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.1),
        layers.RandomContrast([0,1]),
        layers.RandomTranslation(height_factor=0.2, width_factor=0.2)
    ]
)
plt.figure(figsize=(10, 10))
images, _ in train_ds.__getitem__(4)
# for images, _ in train_ds.take(1):
for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow((255*augmented_images[0].numpy()).astype("uint8"))
    plt.axis("off")

## Second Model

In [ ]:
test = labels_image.loc[labels_image['split']=='test']

In [ ]:
test_img

In [ ]:
#Move TEST images to the correct folder, only run once: create a test folder with another test folder with all the images inside
for i in test.index:
    im = cv2.imread(os.path.join(img_folder,test.loc[i,'filename']))
    # cv2.imwrite(test_img + 'test/' + test.loc[i,'filename'], im)
    # im = cv2.imread(os.path.join(img_folder, val1.loc[i, "filename"]))
    cv2.imwrite(os.path.join(test_img, "test", test.loc[i, "filename"]), im)

In [ ]:
# import shutil
# shutil.rmtree(os.path.join(test_img, "test",".ipynb_checkpoints"))

In [ ]:
data.shape

In [ ]:
from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

image = load_img(os.path.join(img_folder,'silos_256-0-0--6-16-536-28464.png'))
image = img_to_array(image).astype(int)
data = np.expand_dims(image, 0)

# Calling ImageDataGenerator for creating data augmentation generator.
datagen = ImageDataGenerator(
    rotation_range=90,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # brightness_range = [0.5, 2.0],
    zca_whitening =True,
    # zoom_range=0.3
)

# Creating an iterator for data augmentation
it = datagen.flow(data, batch_size=1)

# Preparing the Samples and Plot for displaying output
for i in range(9):
    # preparing the subplot
    pyplot.subplot(330 + 1 + i)
    # generating images in batches
    batch = it.next()
    # Remember to convert these images to unsigned integers for viewing 
    image = batch[0].astype('uint8')
    # Plotting the data
    pyplot.imshow(image)

pyplot.show()


In [ ]:
def train_set(train_path, image_size, batch_size):
    """
    Create train dataset to train the model
    Input:
        train_path: path referring to where the train images are stored
        image_size: size of the images
        batch_size: batch size
    Output:
        train_generator: train set in Keras format
    """
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        zca_whitening =True,
        # brightness_range = [0.5, 2.0]
    )

    train_generator = train_datagen.flow_from_directory(
        train_path, target_size=image_size, batch_size=batch_size, class_mode="binary"
    )

    return train_generator


def val_set(val_path, image_size, batch_size):
    """
    Create val dataset to train the model
    Input:
        val_path: path referring to where the val images are stored
        image_size: size of the images
        batch_size: batch size
    Output:
        val_generator: train set in Keras format
    """
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    val_generator = test_datagen.flow_from_directory(
        val_path, target_size=image_size, batch_size=batch_size, class_mode="binary"
    )

    return val_generator


def test_set(test_path, image_size, batch_size):
    """
    Create test dataset to test the model
    Input:
        test_path: path referring to where the test images are stored
        image_size: size of the images
        batch_size: batch size
    Output:
        test_generator: test set in Keras format
    """
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False,
    )

    return test_generator


In [ ]:
## Train, Val, and Test dataset
train_generator = train_set(train_img, config.image_size, config.batch_size)
val_generator = val_set(val_img, config.image_size, config.batch_size)
test_generator = test_set(test_img, config.image_size, config.batch_size)

In [ ]:
## Model
model = keras_model(config.input_shape, train_generator, val_generator, config.batch_size, 2, 'keras_model2')

In [ ]:
# config.input_shape, train_generator, val_generator, config.batch_size, config.number_epochs
model = Sequential()
model.add(Conv2D(64, (2, 2), input_shape=(256,256,3)))
model.add(LeakyReLU(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(LeakyReLU(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(LeakyReLU(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(0.2))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(LeakyReLU(0.2))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"],)

model.fit(
    train_generator,
    steps_per_epoch=1400 // config.batch_size,
    epochs=25,
    validation_data=val_generator,
    validation_steps=400 // config.batch_size,
)

model.save("keras_model2" + ".h5")

In [ ]:
model_keras = load_model("keras_model2.h5")

In [ ]:
preds = test_model(test_generator, model_keras, config.batch_size)
preds

In [ ]:
output_preds(preds, train_generator, test_generator, 'keras_model2_preds')